# Modeling + Forecast
Compare baseline vs improved; export forecasts and metrics.

In [1]:
from __future__ import annotations
import pandas as pd
from pathlib import Path
from typing import Optional

DATA_DIR = Path("Data")

# Expected columns in operations_daily: date, site_id, units_produced, power_kwh, downtime_minutes (optional)
# Expected columns in site_meta: site_id, region, capacity, ...

def read_operations(path: Optional[str] = None, data_dir: Path = DATA_DIR) -> pd.DataFrame:
    REQUIRED_COLS = {"date", "site_id", "units_produced", "power_kwh"}

    def _load(p: Path) -> pd.DataFrame:
        df = pd.read_csv(p, parse_dates=["date"])  # type: ignore[arg-type]
        if not REQUIRED_COLS.issubset(df.columns):
            raise ValueError(f"{p.name} missing required columns: {REQUIRED_COLS - set(df.columns)}")
        return df.sort_values(["site_id", "date"]).reset_index(drop=True)

    if path:
        return _load(Path(path))

    candidates = sorted(data_dir.glob("operations_daily_*.csv"))
    if candidates:
        def _days(p: Path) -> int:
            name = p.stem
            token = name.split("_")[-1]
            return int(token[:-1]) if token.endswith("d") and token[:-1].isdigit() else 0
        chosen = max(candidates, key=_days)
        return _load(chosen)

    fallback = data_dir / "operations_daily.csv"
    if fallback.exists():
        return _load(fallback)

    raise FileNotFoundError("No operations_daily CSV found in Data/")

def read_site_meta(path: Optional[str] = None) -> pd.DataFrame:
    p = Path(path) if path else DATA_DIR / "site_meta.csv"
    if not p.exists():
        raise FileNotFoundError(f"site_meta not found at {p}")
    df = pd.read_csv(p)
    return df

In [2]:
from __future__ import annotations
import pandas as pd
import numpy as np

# Feature engineering utilities
# - Calendar features
# - Rolling means as baselines
# - Site metadata join

def add_calendar_features(df: pd.DataFrame, date_col: str = "date") -> pd.DataFrame:
    d = df.copy()
    d[date_col] = pd.to_datetime(d[date_col])
    d["dow"] = d[date_col].dt.dayofweek  # 0=Mon
    d["dom"] = d[date_col].dt.day
    d["month"] = d[date_col].dt.month
    d["week"] = d[date_col].dt.isocalendar().week.astype(int)
    d["is_weekend"] = (d["dow"] >= 5).astype(int)
    return d


def add_rolling_features(df: pd.DataFrame,
                         by: list[str] = ["site_id"],
                         date_col: str = "date",
                         targets: list[str] = ["units_produced", "power_kwh"],
                         windows: list[int] = [3, 7, 14, 28]) -> pd.DataFrame:
    d = df.copy()
    d = d.sort_values(by + [date_col])
    for tgt in targets:
        if tgt not in d.columns:
            continue
        for w in windows:
            d[f"{tgt}_rollmean_{w}"] = (
                d.groupby(by)[tgt]
                .transform(lambda s: s.rolling(w, min_periods=max(1, w//2)).mean())
            )
            d[f"{tgt}_rollstd_{w}"] = (
                d.groupby(by)[tgt]
                .transform(lambda s: s.rolling(w, min_periods=max(1, w//2)).std())
            )
    return d


def join_site_meta(ops: pd.DataFrame, site_meta: pd.DataFrame) -> pd.DataFrame:
    meta = site_meta.copy()

    # Normalize join key dtype
    ops["site_id"] = ops["site_id"].astype(str)
    meta["site_id"] = meta["site_id"].astype(str)

    # Encode categoricals except join key
    for c in meta.select_dtypes(include=["object"]).columns:
        if c != "site_id":
            try:
                meta[c] = meta[c].astype("category").cat.codes
            except Exception:
                pass

    return ops.merge(meta, on="site_id", how="left")


def prepare_features(ops: pd.DataFrame, site_meta: pd.DataFrame | None = None) -> pd.DataFrame:
    d = add_calendar_features(ops)
    d = add_rolling_features(d)
    if site_meta is not None:
        d = join_site_meta(d, site_meta)
    return d


In [5]:
from __future__ import annotations
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Tuple

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor

# Optional: you can switch to XGBoost/LightGBM if installed
try:
    from xgboost import XGBRegressor  # type: ignore
    HAS_XGB = True
except Exception:
    HAS_XGB = False

HORIZON = 14
TARGETS = ["units_produced", "power_kwh"]


def seasonal_naive_forecast(history: pd.Series, season: int = 7, horizon: int = HORIZON) -> np.ndarray:
    if len(history) == 0:
        return np.zeros(horizon)
    if len(history) < season:
        last = history.iloc[-1]
        return np.repeat(last, horizon)
    template = history.iloc[-season:]
    reps = int(np.ceil(horizon / season))
    fc = np.tile(template.values, reps)[:horizon]
    return fc


@dataclass
class ForecastResult:
    site_id: str | int
    target: str
    dates: List[pd.Timestamp]
    y_true: List[float]
    y_hat_baseline: List[float]
    y_hat_model: List[float]
    mae_baseline: float
    mae_model: float
    mape_baseline: float
    mape_model: float


def _mape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    denom = np.clip(np.abs(y_true), 1e-6, None)
    return float(np.mean(np.abs((y_true - y_pred) / denom)))


def train_improved_model(train_df: pd.DataFrame, feature_cols: List[str], target: str):
    X = train_df[feature_cols].values
    y = train_df[target].values
    if HAS_XGB:
        model = XGBRegressor(
            n_estimators=300,
            max_depth=6,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
        )
    else:
        model = GradientBoostingRegressor(random_state=42)
    model.fit(X, y)
    return model


def rolling_backtest_site(df_site: pd.DataFrame, target: str, feature_cols: List[str], horizon: int = HORIZON,
                          min_train: int = 60) -> Tuple[ForecastResult, pd.DataFrame]:
    d = df_site.sort_values("date").reset_index(drop=True)

    # Train/validation split using expanding window with last horizon as test
    if len(d) < (min_train + horizon):
        # Fallback: train on all but last horizon
        split = max(1, len(d) - horizon)
    else:
        split = len(d) - horizon

    train = d.iloc[:split].copy()
    test = d.iloc[split:].copy()

    # Baseline
    base_fc = seasonal_naive_forecast(train[target], season=7, horizon=len(test))

    # Improved model
    model = train_improved_model(train, feature_cols, target)
    y_hat_model = model.predict(test[feature_cols].values)

    mae_baseline = float(mean_absolute_error(test[target].values, base_fc))
    mae_model = float(mean_absolute_error(test[target].values, y_hat_model))
    mape_baseline = _mape(test[target].values, base_fc)
    mape_model = _mape(test[target].values, y_hat_model)

    res = ForecastResult(
        site_id=d.loc[0, "site_id"],
        target=target,
        dates=list(pd.to_datetime(test["date"])),
        y_true=list(test[target].values.astype(float)),
        y_hat_baseline=list(base_fc.astype(float)),
        y_hat_model=list(y_hat_model.astype(float)),
        mae_baseline=mae_baseline,
        mae_model=mae_model,
        mape_baseline=mape_baseline,
        mape_model=mape_model,
    )

    # Produce next-horizon forecast using all data
    final_model = train_improved_model(d, feature_cols, target)
    # Use last known row features to roll forward naively for horizon days using calendar features only
    future_dates = pd.date_range(d["date"].max() + pd.Timedelta(days=1), periods=horizon, freq="D")
    # Recreate calendar-only features; rolling features won't be available out-of-the-box
    fut = pd.DataFrame({"date": future_dates, "site_id": d.loc[0, "site_id"]})
    fut = add_calendar_features(fut)
    # Fill missing rolling/meta features with last known values per column
    for c in feature_cols:
        if c not in fut.columns:
            last_val = d[c].iloc[-1] if c in d.columns else 0.0
            fut[c] = last_val
    fut = fut[feature_cols]
    future_pred = final_model.predict(fut.values)

    test["y_hat_baseline"] = base_fc
    test["y_hat_model"] = y_hat_model

    return res, pd.DataFrame({
        "site_id": d.loc[0, "site_id"],
        "date": future_dates,
        "target": target,
        "y_hat": future_pred,
    })


def run_modeling(df: pd.DataFrame, feature_cols: List[str]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    forecasts = []
    metrics = []
    for site_id, g in df.groupby("site_id"):
        for tgt in TARGETS:
            if tgt not in g.columns:
                continue
            res, fut = rolling_backtest_site(g, tgt, feature_cols)
            forecasts.append(fut)
            metrics.append({
                "site_id": site_id,
                "target": tgt,
                "mae_baseline": res.mae_baseline,
                "mae_model": res.mae_model,
                "mape_baseline": res.mape_baseline,
                "mape_model": res.mape_model,
            })
    return pd.concat(forecasts, ignore_index=True), pd.DataFrame(metrics)

In [7]:
import pandas as pd

ops = read_operations()
meta = read_site_meta()
X = prepare_features(ops, meta)
feature_cols = [c for c in X.columns if c not in {'date','site_id','units_produced','power_kwh'}]
forecasts, metrics = run_modeling(X, feature_cols)

forecasts_units = forecasts[forecasts['target']=='units_produced'].copy()
forecasts_power = forecasts[forecasts['target']=='power_kwh'].copy()
forecasts_units.to_csv('outputs/forecast_units.csv', index=False)
forecasts_power.to_csv('outputs/forecast_power.csv', index=False)
metrics.to_csv('outputs/model_metrics.csv', index=False)
metrics.head()

,site_id,target,mae_baseline,mae_model,mape_baseline,mape_model
0,S1,units_produced,82.500000,69.132612,0.063631,0.054218
1,S1,power_kwh,561.071429,296.941860,0.112906,0.063655
2,S2,units_produced,152.500000,78.520604,0.100083,0.050412
3,S2,power_kwh,543.000000,309.899693,0.109511,0.062555
4,S3,units_produced,148.214286,56.090947,0.130404,0.050190
